In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
import math

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
import scipy.sparse
import pickle
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.feature_extraction.text import CountVectorizer

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline

D:\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [31]:
df_train = pd.read_csv("train.csv", sep=',')
df_val = pd.read_csv("validation.csv", sep=',')
df_test = pd.read_csv("test.csv", sep=',')

In [5]:
df_train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [6]:
len(df_train.weekday.unique())

7

In [7]:
len(df_train.useragent.unique())

38

In [8]:
len(df_train.region.unique())

35

In [9]:
len(df_train.slotwidth.unique())

21

In [10]:
len(df_train.slotheight.unique())

14

In [11]:
len(df_train.slotvisibility.unique())

11

In [12]:
len(df_train.slotformat.unique())

4

In [13]:
len(df_train.usertag.unique())

744036

In [5]:
df_train.useragent.unique()

array(['windows_ie', 'windows_chrome', 'mac_other', 'windows_other',
       'android_safari', 'mac_safari', 'windows_maxthon',
       'windows_firefox', 'mac_chrome', 'linux_firefox', 'windows_opera',
       'windows_theworld', 'android_other', 'windows_safari', 'ios_safari',
       'linux_other', 'mac_firefox', 'android_opera', 'mac_maxthon',
       'other_other', 'android_chrome', 'linux_chrome', 'other_opera',
       'linux_safari', 'other_ie', 'ios_other', 'linux_opera',
       'android_sogou', 'other_safari', 'windows_sogou', 'mac_opera',
       'mac_sogou', 'android_maxthon', 'android_firefox', 'linux_ie',
       'other_chrome', 'android_ie', 'other_firefox'], dtype=object)

# Data Processing

In [21]:
#One hot variables
def pipeline(data):
    one_hot_features = ['useragent','adexchange','slotvisibility','slotformat']
    new_df = pd.get_dummies(data[one_hot_features])
    
    weekday = pd.get_dummies(data["weekday"], prefix='weekday_')
    hour = pd.get_dummies(data["hour"], prefix='hour_')
    region = pd.get_dummies(data["region"], prefix='region_')
    city = pd.get_dummies(data["city"], prefix='city_')
    slotwidth = pd.get_dummies(data["slotwidth"], prefix='slotwidth_')
    slotheight = pd.get_dummies(data["slotheight"], prefix='slotheight_')
    advertiser = pd.get_dummies(data['advertiser'], prefix='advertiser_')

    new_df = pd.concat([new_df,data['slotprice'], weekday, hour, region,city,slotwidth, slotheight, advertiser],axis=1)
    
    labelencoder_X_1 = LabelEncoder()
    new_df.useragent = labelencoder_X_1.fit_transform(data.useragent)
    labelencoder_X_2 = LabelEncoder()
    new_df.slotformat = labelencoder_X_2.fit_transform(data.slotformat)
    labelencoder_X_3 = LabelEncoder()
    new_df.usertag = labelencoder_X_3.fit_transform(data.usertag)
    
    new_df = pd.concat([new_df,data.click],axis=1)
    
    
    return new_df

In [22]:
df_train = pipeline(df_train)

In [23]:
df_train.shape

(2430981, 540)

In [24]:
df_val = pipeline(df_val)

In [25]:
df_val.shape

(303925, 537)

In [32]:
#One hot variables
def pipeline2(data):
    one_hot_features = ['useragent','adexchange','slotvisibility','slotformat']
    new_df = pd.get_dummies(data[one_hot_features])
    
    weekday = pd.get_dummies(data["weekday"], prefix='weekday_')
    hour = pd.get_dummies(data["hour"], prefix='hour_')
    region = pd.get_dummies(data["region"], prefix='region_')
    city = pd.get_dummies(data["city"], prefix='city_')
    slotwidth = pd.get_dummies(data["slotwidth"], prefix='slotwidth_')
    slotheight = pd.get_dummies(data["slotheight"], prefix='slotheight_')
    advertiser = pd.get_dummies(data['advertiser'], prefix='advertiser_')

    new_df = pd.concat([new_df,data['slotprice'], weekday,city, hour, region, slotwidth, slotheight, advertiser],axis=1)
    
    labelencoder_X_1 = LabelEncoder()
    new_df.useragent = labelencoder_X_1.fit_transform(data.useragent)
    labelencoder_X_2 = LabelEncoder()
    new_df.slotformat = labelencoder_X_2.fit_transform(data.slotformat)
    labelencoder_X_3 = LabelEncoder()
    new_df.usertag = labelencoder_X_3.fit_transform(data.usertag)
    

    return new_df

In [33]:
df_test = pipeline2(df_test)

In [34]:
df_test.shape

(303375, 534)

In [35]:
df_train.head()

,useragent_android_chrome,useragent_android_firefox,useragent_android_ie,useragent_android_maxthon,useragent_android_opera,useragent_android_other,useragent_android_safari,useragent_android_sogou,useragent_ios_other,useragent_ios_safari,...,advertiser__1458,advertiser__2259,advertiser__2261,advertiser__2821,advertiser__2997,advertiser__3358,advertiser__3386,advertiser__3427,advertiser__3476,click
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [36]:
# Features list of all three datasets
train_features = list(df_train)
train_features.remove('click')
val_features = list(df_val)
val_features.remove('click')
test_features = list(df_test)

In [37]:
print('No. of features in training set', len(train_features))
print('No. of features in validation set', len(val_features))
print('No. of features in test set', len(test_features))

No. of features in training set 539
No. of features in validation set 536
No. of features in test set 534


In [38]:
from collections import Counter
print('Whether the train and val have the same one-hot features?')
print(Counter(train_features) == Counter(val_features))
if Counter(train_features) != Counter(val_features):
    print(' ')
    print('Missing features:')
    print(list(set(train_features) - set(val_features)))

Whether the train and val have the same one-hot features?
False
 
Missing features:
['useragent_other_firefox', 'useragent_android_maxthon', 'useragent_android_ie']


In [39]:
print('Whether the train and test have the same one-hot features?')
print(Counter(train_features) == Counter(test_features))
if Counter(train_features) != Counter(test_features):
    print(' ')
    print('Missing features:')
    print(list(set(train_features) - set(test_features)))

Whether the train and test have the same one-hot features?
False
 
Missing features:
['useragent_mac_sogou', 'useragent_mac_maxthon', 'useragent_other_firefox', 'useragent_linux_ie', 'useragent_android_ie']


In [40]:
# Final features selection for train and validation

features = list(df_train)
remove_feature = ['bidprice', 'payprice', 'useragent_android_ie', 'useragent_other_firefox', 'useragent_android_maxthon',
              'useragent_linux_ie', 'useragent_mac_sogou', 'useragent_other_firefox', 'useragent_android_ie', 'useragent_mac_maxthon']
feature_new = [feature for feature in features if (feature not in remove_feature)]

In [41]:
train_sample = df_train.sample(frac=0.3, random_state = 2)

In [21]:
train_sample.groupby('click').count()

,useragent_android_chrome,useragent_android_firefox,useragent_android_ie,useragent_android_maxthon,useragent_android_opera,useragent_android_other,useragent_android_safari,useragent_android_sogou,useragent_ios_other,useragent_ios_safari,...,slotheight__600,advertiser__1458,advertiser__2259,advertiser__2261,advertiser__2821,advertiser__2997,advertiser__3358,advertiser__3386,advertiser__3427,advertiser__3476
click,,,,,,,,,,,,,,,,,,,,,
0,728758,728758,728758,728758,728758,728758,728758,728758,728758,728758,...,728758,728758,728758,728758,728758,728758,728758,728758,728758,728758
1,536,536,536,536,536,536,536,536,536,536,...,536,536,536,536,536,536,536,536,536,536


In [42]:
X_train = train_sample[feature_new]
y_train = train_sample['click']
X_val = df_val[feature_new]
y_val = df_val['click']

In [43]:
features2 = list(df_test)
remove_feature = ['bidprice', 'payprice', 'useragent_android_ie', 'useragent_other_firefox', 'useragent_android_maxthon',
              'useragent_linux_ie', 'useragent_mac_sogou', 'useragent_other_firefox', 'useragent_android_ie', 'useragent_mac_maxthon']
feature_new2 = [feature for feature in features2 if (feature not in remove_feature)]

In [44]:
X_test = df_test[feature_new2]

In [45]:
X_test.shape

(303375, 533)

In [46]:
X_train.shape

(729294, 534)

In [49]:
Counter(X_train.click)

Counter({0: 728758, 1: 536})

In [50]:
Counter(X_val.click)

Counter({0: 303723, 1: 202})

# Random forest

In [51]:
rf_model = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=5, min_samples_split=2, \
                                  min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', \
                                  max_leaf_nodes=None, bootstrap=True, oob_score=False, \
                                  n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [54]:
# Result based on probability of clicking
prob = []
for a,b in rf_model.predict_proba(X_val):
    prob.append(b)

#y_pred_rf_prob = prob
#auc = roc_auc_score(y_val, y_pred_rf_prob)
rmse = math.sqrt(mean_squared_error(y_val, y_pred_rf_prob))
#print('AUC:', auc, 'RMSE:',rmse)
print ('RMSE', rmse)

RMSE 0.021890811022771205


In [42]:
pCTR = pd.DataFrame(prob)

In [43]:
pCTR.to_csv("RF result.csv")

In [50]:
pCTR2 = pd.DataFrame(prob2)

In [51]:
pCTR2.to_csv("RF result2.csv")

# ExtraTree Classifier

In [52]:
etc_model_full = ExtraTreesClassifier(n_estimators=1000, criterion='gini', max_depth=50, min_samples_split=2,\
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', \
                                 max_leaf_nodes=None, bootstrap=False, oob_score=False, n_jobs=1, random_state=None, verbose=0, \
                                 warm_start=False, class_weight=None)
etc_model_full.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=50, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [53]:
# Probability
etc_full_pred = []
for a, b in etc_model_full.predict_proba(X_val):
    etc_full_pred.append(b)

#auc_etcf = roc_auc_score(df_val['click'], etc_full_pred)
rmse_etcf = math.sqrt(mean_squared_error(df_val['click'], etc_full_pred))
#print('AUC:', auc_etcf, 'RMSE:',rmse_etcf)

NameError: name 'etc_model_full' is not defined

In [57]:
pCTR_ex = pd.DataFrame(etc_full_pred)

In [ ]:
pCTR_ex.to_csv("result extratree.csv")